In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from vnstock import Quote
from vnstock import Finance
import time

<trans style = "font-family:Times New Roman; font-size:30px;color:red">
Thu thap du lieu

<trans style="font-family:Times New Romans; font-size: 25px; color:blue">
Xac dinh cac cong ty can de tam

Phan nay khong chay lai nua

In [ ]:
#Tim ma chung khoan cua cac cong ty
symbol = pd.read_csv('Du lieu.csv')
symbol.columns = symbol.iloc[0]
symbol = symbol.iloc[1::]
interest_industry = list(symbol['Lĩnh vực'].unique())
interest_industry.pop(1)
symbol = symbol[symbol['Lĩnh vực'].isin(interest_industry)]
symbol_name = list(symbol['Công ty'])

In [ ]:
#Phan chia so lan lay du lieu
STT_list = [0,55,110,156]
name_bins = {}
for i in range(len(STT_list)):
    if STT_list[i] == 0:
        continue
    else:
        name_bins[STT_list[i]] = symbol_name[STT_list[i-1]:STT_list[i]]
history_price = pd.DataFrame()

In [ ]:
#Thu thap du lieu ve gia
#Khong can chay o du lieu nay
for i in name_bins[156]:
    pr = Quote(source='VCI',symbol=i)
    price_status = pr.history(start='2010-01-01',end='2025-01-01',interval='1M')
    price_status['time'] = pd.to_datetime(price_status['time'])
    price_status.set_index('time', inplace=True)
    history_price[i] = price_status['close']

<trans style="font-family:Times News Romans; font-size:25px;color:blue">
Chon loc ra 100 co phieu hoac it hon trong tung giai doan

Khong chay lai doan code nay nx

In [ ]:
#chia co phieu ra lam 2 thoi ky
history_price = pd.read_csv('price data.csv')
history_price['time'] = pd.to_datetime(history_price['time'])
history_price.set_index('time', inplace=True)
history_price = history_price[history_price.index.year>2015]
history_price.dropna(axis=1,inplace=True)
history_price.to_csv('compare_period1.csv')


<trans style="font-family:Times News Roman;font-size:25px; color: blue">
Tai du lieu cua bao cao tai chinh ve

Khong can chay lai nx

In [ ]:
#Cac co phieu can lay bao cao tai chinh
data = pd.read_csv('compare_period1.csv')
data['time'] = pd.to_datetime(data['time'])
data.set_index('time',inplace=True)
stock_of_interest = list(data.columns)

In [ ]:
#chia ra thanh cac muc nho hon de lay du lieu
count = 0
stock_trench = {}
number_stock_per_trench = 7
number_of_trench = int(round(len(stock_of_interest)/number_stock_per_trench,0)+1)
separate_support = [min(number_stock_per_trench*i, len(stock_of_interest)) for i in range(number_of_trench)]#cu chon  co phieu 1 danh sach
for i in range(1,len(separate_support)):
    stock_trench[separate_support[i]] = stock_of_interest[separate_support[i-1]:separate_support[i]]
financial_statement_data = pd.DataFrame()#phuc vu cho cell ke tiep
stock_trench

In [ ]:
char_cf = ['Năm', 'Kỳ','Khấu hao TSCĐ',
           'Mua sắm TSCĐ','Tiền thu được từ thanh lý tài sản cố định'  ]
char_is = ['Năm', 'Kỳ','Doanh thu thuần','Thu nhập tài chính','Thu nhập khác','LN trước thuế','Chi phí tiền lãi vay']
char_bs = ['Năm', 'Kỳ','TÀI SẢN NGẮN HẠN (đồng)','Tiền và tương đương tiền (đồng)', 'Giá trị thuần đầu tư ngắn hạn (đồng)','Nợ ngắn hạn (đồng)',
           'Vay và nợ thuê tài chính ngắn hạn (đồng)',
           'Vay và nợ thuê tài chính dài hạn (đồng)',
           'TỔNG CỘNG TÀI SẢN (đồng)','VỐN CHỦ SỞ HỮU (đồng)','LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ']
char_ratio = ['Năm', 'Kỳ','Số CP lưu hành (Triệu CP)']
sleep_time = 40
for t in list(stock_trench.keys())[-2::]:
    for i in stock_trench[t]:
        if i == "DPP":
            continue
        else:
            finance = Finance(source='VCI',symbol=i)
            #bao cao ket qua hoat dong kinh doanh
            income_state = finance.income_statement(lang='vi')
            for j in char_is:
                if j not in income_state.columns:
                    income_state[j]=0
            income_state = income_state[char_is]
            income_state = income_state[income_state['Năm']>2012]
            year = list(income_state['Năm'])
            quarter = list(income_state['Kỳ'])
            time1 = [f'{i}Q{j}' for i,j in zip(year,quarter)]
            time1 = pd.PeriodIndex(time1,freq='Q').to_timestamp(how='end').normalize()
            income_state['time'] = time1
            income_state.set_index('time',inplace=True)
            income_state.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #bang can doi ke toan
            balance_sheet = finance.balance_sheet(lang='vi')
            for j in char_bs:
                if j not in balance_sheet.columns:
                    balance_sheet[j]=0
            balance_sheet = balance_sheet[char_bs]
            balance_sheet = balance_sheet[balance_sheet['Năm']>2012]
            balance_sheet['time'] = time1
            balance_sheet.drop(['Năm','Kỳ'],axis=1, inplace=True)
            balance_sheet.set_index('time', inplace=True)
            #bao cao luu chuyen tien te
            cashflow = finance.cash_flow(lang='vi')
            for j in char_cf:
                if j not in cashflow.columns:
                    cashflow[j] = 0
            cashflow = cashflow[char_cf]
            cashflow = cashflow[cashflow['Năm']>2012]
            cashflow['time'] = time1
            cashflow.set_index('time', inplace=True)
            cashflow.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #so luong co phieu dang luu hanh
            outstanding_share = finance.ratio(lang='vi')
            outstanding_share = outstanding_share.droplevel(level=0,axis=1)
            for j in char_ratio:
                if j not in outstanding_share.columns:
                    outstanding_share[j] = 0
            outstanding_share = outstanding_share[char_ratio]
            outstanding_share = outstanding_share[outstanding_share['Năm']>2012]
            outstanding_share['time']=time1
            outstanding_share.set_index('time',inplace=True)
            outstanding_share.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #sap nhap cac bao cao lai thanh 1 dataframe
            financial_statement_subdata = pd.concat([income_state,balance_sheet,cashflow,outstanding_share], axis=1)
            financial_statement_subdata['ticket'] = i
            #sap nhap cac cong ty lai thanh 1
            if len(financial_statement_data) == 0:
                financial_statement_data = financial_statement_subdata
            else:
                financial_statement_data = pd.concat([financial_statement_data,financial_statement_subdata],axis=0)
    time.sleep(sleep_time)


<trans style = "font-family:Calibri;font-size:25px;color:blue">
Xac nhan lai cac cong ty can quan tam va thu thap lai du lieu ve gia

In [ ]:
#lay du lieu
financial_statement_data = pd.read_csv('financial_statements.csv')
financial_statement_data['time'] = pd.to_datetime(financial_statement_data['time'])
financial_statement_data.set_index(['ticket','time'], inplace=True)

In [35]:
#Phan trench de lay du lieu
stock_of_interest = list(financial_statement_data.index.get_level_values(0).unique())
stock_trench = {}
number_stock_per_trench = 20
number_of_trench = int(round(len(stock_of_interest)/number_stock_per_trench,0)+1)
trench = [min(number_stock_per_trench*i,len(stock_of_interest)) for i in range(number_of_trench)]
l=0
for i in range(1,len(trench)):
    stock_trench[trench[i]] = stock_of_interest[trench[i-1]:trench[i]]
    l += len(stock_of_interest[trench[i-1]:trench[i]])



In [ ]:
input_data = pd.DataFrame()
compare_data = pd.DataFrame()
for t in stock_trench.keys():
    for i in stock_trench[t]:
        his = Quote(source='VCI', symbol=i)
        #du lieu dau vao
        input_subdata = his.history(start='2015-01-01',end='2020-01-01',interval='1M')
        input_subdata['time'] = pd.to_datetime(input_subdata['time'])
        input_subdata.set_index('time',inplace=True)
        input_subdata = input_subdata[['close']]
        input_subdata.columns = [i]
        #Du lieu so sanh
        compare_subdata = his.history(start='2020-01-01',end='2026-01-01',interval='1D')
        compare_subdata['time'] = pd.to_datetime(compare_subdata['time'])
        compare_subdata.set_index('time',inplace=True)
        compare_subdata = compare_subdata[['close']]
        compare_subdata.columns = [i]
        if len(input_data)==0:
            input_data = input_subdata
        else:
            input_data = pd.concat([input_data,input_subdata],axis=1)
        if len(compare_data)==0:
            compare_data = compare_subdata
        else:
            compare_data = pd.concat([compare_data,compare_subdata],axis=1)
    time.sleep(35)